In [1]:
import numpy as np
from boxstats import normalize_box_shape, to_numpy_iter

bbox = [[[5,7,15,32],[65,43,15,42]], [[50,50,25,25]], [[14,8,32,32], [34,23,49,67], [42,31,36,47]], [14,19,58,59]]
for boxes in to_numpy_iter(bbox):
    norm_boxes = normalize_box_shape(boxes)
    print(norm_boxes.shape)
    for box in norm_boxes:
        print(f"\t{box}")

2024-09-20 19:55:08.669474: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-20 19:55:08.671541: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-20 19:55:08.695848: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-20 19:55:08.695872: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-20 19:55:08.696617: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

(2, 4)
	[ 5  7 15 32]
	[65 43 15 42]
(1, 4)
	[50 50 25 25]
(3, 4)
	[14  8 32 32]
	[34 23 49 67]
	[42 31 36 47]
(1, 4)
	[14 19 58 59]


In [2]:
img1 = np.ones((1,100,100))
img1[0,7:67,10:75] += np.random.random((60,65))
img2 = np.ones((1,75,75))
img2[0,50:75,25:75] += 5*np.random.random((25,50))
img3 = np.zeros((1,100,100))
img3[0,20:70,:] += 5*np.random.random((50,100))
img4 = np.zeros((1,90,90))
img4[0,20:60,20:60] += np.random.random((40,40))

images = [img1, img2, img3, img4]

In [3]:
from dataeval._internal.metrics.stats import imagestats
from boxstats import boxstats

img_results = imagestats(images)
box_results = boxstats(bbox, images)

In [4]:
result = []
result2 = []
for boxes, image in zip(to_numpy_iter(bbox), to_numpy_iter(images)):
    norm_boxes = normalize_box_shape(boxes)
    boxes_output = []
    box_out = []
    for box in norm_boxes:
        box_image = image[:, box[1] : box[1] + box[3], box[0] : box[0] + box[2]]
        boxes_output.append(np.asarray([np.uint16(box[0]), np.uint16(box[1])]))
        box_out.append(np.float16(np.percentile(box_image, q=(0, 25, 50, 75, 100))))
    result.append(np.asarray(boxes_output))
    result2.append(np.asarray(box_out))
print(result)
print(result2)


[array([[ 5,  7],
       [65, 43]], dtype=uint16), array([[50, 50]], dtype=uint16), array([[14,  8],
       [34, 23],
       [42, 31]], dtype=uint16), array([[14, 19]], dtype=uint16)]
[array([[1.   , 1.   , 1.253, 1.589, 1.989],
       [1.   , 1.   , 1.   , 1.355, 2.   ]], dtype=float16), array([[1.008, 2.363, 3.549, 4.805, 5.99 ]], dtype=float16), array([[0.    , 0.    , 1.066 , 3.035 , 4.96  ],
       [0.    , 0.    , 1.501 , 3.252 , 5.    ],
       [0.    , 0.5264, 2.107 , 3.56  , 5.    ]], dtype=float16), array([[0.    , 0.    , 0.    , 0.4778, 0.9995]], dtype=float16)]


In [5]:
box_results.box_count

array([2, 1, 3, 1], dtype=uint16)

In [6]:
box_results

StatsBoxOutput(box_count=array([2, 1, 3, 1], dtype=uint16), center=array([[[10, 19],
        [40, 42],
        [ 0,  0]],

       [[37, 37],
        [ 0,  0],
        [ 0,  0]],

       [[23, 20],
        [41, 45],
        [39, 39]],

       [[36, 39],
        [ 0,  0],
        [ 0,  0]]], dtype=uint16), width=array([[15, 15,  0],
       [25,  0,  0],
       [32, 49, 36],
       [58,  0,  0]], dtype=uint16), height=array([[32, 42,  0],
       [25,  0,  0],
       [32, 67, 47],
       [59,  0,  0]], dtype=uint16), channels=array([], dtype=float64), size=array([[ 480,  630,    0],
       [ 625,    0,    0],
       [1024, 3283, 1692],
       [3422,    0,    0]], dtype=uint32), aspect_ratio=array([[0.4688, 0.3572,    nan],
       [1.    ,    nan,    nan],
       [1.    , 0.7314, 0.766 ],
       [0.983 ,    nan,    nan]], dtype=float16), depth=array([], dtype=float64), brightness=array([[  6.48 ,   6.48 ,     nan],
       [303.2  ,     nan,     nan],
       [312.5  , 312.5  , 312.5  ],
    

In [7]:
image_width = np.array([100, 75, 100, 90])
image_height = np.array([100, 75, 100, 90])
image_center = np.stack([image_width/2, image_height/2], axis=1)
image_center

array([[50. , 50. ],
       [37.5, 37.5],
       [50. , 50. ],
       [45. , 45. ]])

In [8]:
box_results.center[:, :, 0]

array([[10, 40,  0],
       [37,  0,  0],
       [23, 41, 39],
       [36,  0,  0]], dtype=uint16)

In [9]:
box_results.center[:, :, 1]

array([[19, 42,  0],
       [37,  0,  0],
       [20, 45, 39],
       [39,  0,  0]], dtype=uint16)

In [10]:
width_ratio = (box_results.center[:, :, 0] - image_width[:, np.newaxis] / 2) / (
    image_width[:, np.newaxis] / 2
)
height_ratio = (image_height[:, np.newaxis] / 2 - box_results.center[:, :, 1]) / (
    image_height[:, np.newaxis] / 2
)
np.stack([width_ratio, height_ratio], axis=2)

array([[[-0.8       ,  0.62      ],
        [-0.2       ,  0.16      ],
        [-1.        ,  1.        ]],

       [[-0.01333333,  0.01333333],
        [-1.        ,  1.        ],
        [-1.        ,  1.        ]],

       [[-0.54      ,  0.6       ],
        [-0.18      ,  0.1       ],
        [-0.22      ,  0.22      ]],

       [[-0.2       ,  0.13333333],
        [-1.        ,  1.        ],
        [-1.        ,  1.        ]]])

In [11]:
from boxstats import box_image_ratio_stats

box_image_ratio_stats(box_results, img_results)

/workspaces/dataeval/prototype/boxstats.py:507: RuntimeWarning: overflow encountered in cast
  output[result] = (box_stats[stat] / image_divide[:, np.newaxis]).astype(np.float16)


StatsRatioOutput(density=array([0.111 , 0.1111, 0.6   , 0.4224], dtype=float16), count_density=array([18.02 ,  9.   ,  5.   ,  2.367], dtype=float16), center_ratio=array([[[-0.8     ,  0.62    ],
        [-0.2     ,  0.16    ],
        [-1.      ,  1.      ]],

       [[-0.013336,  0.013336],
        [-1.      ,  1.      ],
        [-1.      ,  1.      ]],

       [[-0.54    ,  0.6     ],
        [-0.18    ,  0.1     ],
        [-0.22    ,  0.22    ]],

       [[-0.2     ,  0.1333  ],
        [-1.      ,  1.      ],
        [-1.      ,  1.      ]]], dtype=float16), width_ratio=array([[0.15  , 0.15  , 0.    ],
       [0.3333, 0.    , 0.    ],
       [0.32  , 0.49  , 0.36  ],
       [0.6445, 0.    , 0.    ]], dtype=float16), height_ratio=array([[0.32  , 0.42  , 0.    ],
       [0.3333, 0.    , 0.    ],
       [0.32  , 0.67  , 0.47  ],
       [0.656 , 0.    , 0.    ]], dtype=float16), size_ratio=array([[0.048 , 0.063 , 0.    ],
       [0.1111, 0.    , 0.    ],
       [0.1024, 0.3284, 0.16